In [ ]:
#Verificar si tienes GPU para mayor velocidad de entrenamiento
!nvidia-smi

In [ ]:
#####
# 1 #
#####
# INSTALAR DEPENDENCIAS DE PIPER #
!apt-get update
!apt-get install -y --fix-missing git unzip wget
%cd /kaggle/working
!git clone -q https://github.com/rmcpantoja/piper
%cd /kaggle/working/piper/src/python
!wget -q "https://raw.githubusercontent.com/coqui-ai/TTS/dev/TTS/bin/resample.py"
!rm -rf /opt/conda/lib/python3.10/site-packages/aiohttp-3.9.1.dist-info
!rm -rf /opt/conda/lib/python3.10/site-packages/numpy-1.26.4.dist-info
!pip install numpy gdown
!pip install openai-whisper
#!pip install -q -r requirements.txt
!pip install -q cython>=0.29.0 piper-phonemize==1.1.0 librosa>=0.9.2 numpy==1.24 onnxruntime>=1.11.0 pytorch-lightning==1.7.7 torch==1.13.0+cu117 --extra-index-url https://download.pytorch.org/whl/cu117
!pip install -q torchtext==0.14.0 torchvision==0.14.0
!python3 -m pip install --no-binary numpy numpy==1.21.5
!python3 -m pip install --force-reinstall numpy
#!pip install -U openai-whisper
# fixing recent compativility issues:
!pip install -q torchaudio==0.13.0 torchmetrics==0.11.4 faster_whisper
!bash build_monotonic_align.sh
# Useful vars:
use_whisper = True
print("Done!")

In [ ]:
#####
# 2 #
#####
# IMPORTAR DATASET VOCES.ZIP Y ARCHIVO DE TRANSCRIPCION
import os
# Variables para los enlaces de Google Drive
wavs_id = input("Por favor ingresa el ID de Google Drive para el archivo de wavs: ")
metadata_id = input("Por favor ingresa el ID de Google Drive para el archivo de metadata: ")

# Instalamos gdown si no está instalado
!pip install gdown; apt install unzip -y

# Creamos el directorio si no existe
dataset_dir = "/kaggle/working/piper/my-dataset"
wavs_dir = os.path.join(dataset_dir, "wavs")

if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir)

if not os.path.exists(wavs_dir):
    os.makedirs(wavs_dir)

# Descargamos los archivos usando los IDs proporcionados
!gdown --id {wavs_id} -O {wavs_dir}/wavs.zip
!gdown --id {metadata_id} -O {dataset_dir}/metadata.csv

# Descomprimimos los archivos
%cd {wavs_dir}
!unzip wavs.zip
!rm wavs.zip



In [16]:
%cd /kaggle/working/piper/my-training/lightning_logs/version_1/checkpoints/
!rm 'epoch=10375-step=1755104.ckpt'

/kaggle/working/piper/my-training/lightning_logs/version_1/checkpoints


In [ ]:
#####
# 3 #
#####
# PREPARAR DATASET
%cd /kaggle/working/piper/src/python
!python3 -m piper_train.preprocess \
  --language es-419 \
  --input-dir /kaggle/working/piper/my-dataset \
  --output-dir /kaggle/working/piper/my-training \
  --dataset-format ljspeech \
  --single-speaker \
  --sample-rate 22050

In [ ]:
#####
# 4 #
#####
############################################
# EMPEZAR A ENTRENAR/CONTINUAR ENTRENANDO #
###########################################
# EMPIEZA A ENTRENAR TU DATASET CON EL MODELO BASE O SI ENCUENTRA 
# UN PUNTO DE ENTRENAMIENTO ACTUAL SIGUE CON EL ENTRENAMIENTO
%cd /kaggle/working/piper/src/python
script_content = """
# Buscar el último checkpoint en lightning_logs
latest_checkpoint=$(ls -1 /kaggle/working/piper/my-training/lightning_logs | grep version_ | sort -t_ -k2,2n | tail -n 1)
latest_checkpoint_dir="/kaggle/working/piper/my-training/lightning_logs/$latest_checkpoint/checkpoints"

# Encontrar el último archivo de checkpoint
latest_checkpoint_file=$(ls -1 "$latest_checkpoint_dir" | grep 'epoch=[0-9]*-step=[0-9]*\.ckpt' | sort -t= -k2,2n | tail -n 1)

# Verificar si se encontró algún checkpoint
if [ -n "$latest_checkpoint_file" ]; then
    full_checkpoint_path="$latest_checkpoint_dir/$latest_checkpoint_file"
    echo "Último checkpoint encontrado: $full_checkpoint_path"
else
    # Si no se encuentra, descargar el modelo de checkpoint
    download_url="https://huggingface.co/datasets/rhasspy/piper-checkpoints/resolve/main/es/es_ES/davefx/medium/epoch%3D5629-step%3D1605020.ckpt"
    save_path="/kaggle/working/piper/last.ckpt"
    wget "$download_url" -O "$save_path"
    full_checkpoint_path="$save_path"
    echo "No se encontraron checkpoints. Descargando modelo en: $full_checkpoint_path"

    # Ejecutar el código nuevamente con el modelo descargado

    python3 -m piper_train \
        --dataset-dir /kaggle/working/piper/my-training \
        --accelerator 'gpu' \
        --devices 1 \
        --batch-size 12 \
        --validation-split 0.0 \
        --num-test-examples 0 \
        --max_epochs 20000 \
        --resume_from_checkpoint "$full_checkpoint_path" \
        --checkpoint-epochs 5 \
        --precision 32
    exit
fi

# Ejecutar el comando piper_train con el último checkpoint
python3 -m piper_train \
    --dataset-dir /kaggle/working/piper/my-training \
    --accelerator 'gpu' \
    --devices 1 \
    --batch-size 12 \
    --validation-split 0.0 \
    --num-test-examples 0 \
    --max_epochs 20000 \
    --resume_from_checkpoint "$full_checkpoint_path" \
    --checkpoint-epochs 5 \
    --precision 32

"""

# Guarda el script en un archivo temporal
script_file = "/kaggle/working/piper/src/python/run_piper_train.sh"
with open(script_file, "w") as f:
    f.write(script_content)

# Ejecuta el script
!bash {script_file}


# [Cuaderno de Inference Colab para probar modelos (No requiere CPU)](https://colab.research.google.com/github/rmcpantoja/piper/blob/master/notebooks/piper_inference_(ckpt).ipynb)
# [Exportar modelos a .onnx para usarlos en cualquier sistema operativo, Windows, Linux, etc.](https://colab.research.google.com/github/rmcpantoja/piper/blob/master/notebooks/piper_model_exporter.ipynb)

In [25]:
######################################################################################
## GENERAR LINKS DE DESCARGA DE LOS ARCHIVOS CKPT Y JSON PARA PROBARLOS DESDE COLAB ##
######################################################################################

script_content = """
# Obtener el último checkpoint
latest_checkpoint=$(ls -1 /kaggle/working/piper/my-training/lightning_logs | grep version_ | sort -t_ -k2,2n | tail -n 1)
latest_checkpoint_dir="/kaggle/working/piper/my-training/lightning_logs/$latest_checkpoint/checkpoints"
latest_checkpoint_file=$(ls -1 "$latest_checkpoint_dir" | grep 'epoch=[0-9]*-step=[0-9]*\.ckpt' | sort -t= -k2,2n | tail -n 1)
full_checkpoint_path="$latest_checkpoint_dir/$latest_checkpoint_file"
# Ruta al archivo config.json
config_json_path="/kaggle/working/piper/my-training/config.json"
# Subir el último checkpoint a bashupload.com
upload_checkpoint=$(curl -s https://bashupload.com/ -T "$full_checkpoint_path")
checkpoint_download_url=$(echo "$upload_checkpoint" | grep -o 'https://bashupload.com/[^ ]*')
echo "Uploaded checkpoint. Download URL: $checkpoint_download_url"
# Subir config.json a bashupload.com
upload_config=$(curl -s https://bashupload.com/ -T "$config_json_path")
config_download_url=$(echo "$upload_config" | grep -o 'https://bashupload.com/[^ ]*')
echo "Uploaded config.json. Download URL: $config_download_url"
"""

# Guarda el script en un archivo temporal
script_file = "/kaggle/working/piper/src/python/generate-inference-links-files.sh"
with open(script_file, "w") as f:
    f.write(script_content)

# Ejecuta el script
!bash {script_file}

Uploaded checkpoint. Download URL: https://bashupload.com/wnVar/yRTzm.ckpt
Uploaded config.json. Download URL: https://bashupload.com/peLDP/config.json


In [ ]:
# En caso de tener errores similares a formato comprimido del modelo
%cd /kaggle/working/piper/my-training/lightning_logs/version_3/checkpoints
!ls
!rm 'epoch=10741-step=1756568.ckpt'

In [ ]:
#####################################################################
# DESCARGAR PROGRESO DE ENTRENAMIENTO (archivo .ckpt y config.json) #
#####################################################################
# DESCARGA UN COMPRIMIDO CON EL PROGRESO DE TU ENTRENAMIENDO

script_content = """
# Verifica la existencia de carpetas en lightning_logs
logs_exist=$(ls -1 /kaggle/working/piper/my-training/lightning_logs | grep version_ | sort -t_ -k2,2n | tail -n 1)

if [ -n "$logs_exist" ]; then
    # Si existen carpetas en lightning_logs, utiliza el último checkpoint
    latest_checkpoint_dir="/kaggle/working/piper/my-training/lightning_logs/$logs_exist/checkpoints"
    latest_checkpoint_file=$(ls -1 "$latest_checkpoint_dir" | grep 'epoch=[0-9]*-step=[0-9]*\.ckpt' | sort -t= -k2,2n | tail -n 1)
    full_checkpoint_path="$latest_checkpoint_dir/$latest_checkpoint_file"
else
    # Si no existen carpetas, verifica la existencia del archivo last.ckpt
    if [ -e "/kaggle/working/piper/last.ckpt" ]; then
        full_checkpoint_path="/kaggle/working/piper/last.ckpt"
    else
        echo "No se encontraron carpetas en lightning_logs y no hay archivo last.ckpt. No se puede reanudar el entrenamiento."
        exit 1
    fi
fi

# Verifica si se encontró un checkpoint válido
if [ -n "$full_checkpoint_path" ]; then
    # Verifica la existencia del archivo config.json
    config_file="/kaggle/working/piper/my-training/config.json"
    
    if [ -e "$config_file" ]; then
        echo "Ambos archivos encontrados. Creando el archivo comprimido con nombre modificado..."
        
        # Obtiene el nombre del archivo sin el path
        checkpoint_filename=$(basename "$full_checkpoint_path")
        
        # Sustituye '=' por '-' en el nombre del archivo comprimido
        compressed_filename=$(echo "$checkpoint_filename" | tr '=' '-')
        
        # Crea el archivo comprimido con el nombre modificado
        tar -czvf "$compressed_filename.gz" -C $(dirname "$full_checkpoint_path") "$checkpoint_filename" "$config_file"
        
        # Sube el archivo a bashupload.com usando curl
        echo "Subiendo el archivo a bashupload.com..."
        curl --upload-file "$compressed_filename.gz" https://bashupload.com/
        
        echo "Subida completada."
        
        # Elimina el archivo comprimido después de la subida
        rm "$compressed_filename.gz"
    else
        echo "No se encontró el archivo $config_file. No se puede crear el archivo comprimido."
    fi
else
    echo "No se encontró un checkpoint válido. El archivo comprimido no se creará ni se subirá."
fi
"""

# Guarda el script en un archivo temporal
script_file = "/kaggle/working/piper/src/python/backup-checkpoint.sh"
with open(script_file, "w") as f:
    f.write(script_content)

# Ejecuta el script
!bash {script_file}